# I. Main MELD Script

In [1]:
import pandas as pd
import numpy as np

def calcMELD (Creatinine, renal_comorb, Bilirubin, INR, Na):
    """
    adapted from: https://github.com/9929105/meld-cal/blob/master/meld-cal.ipynb
    
    With the following Variables

    Creatinine (mg/dL) Norm = 0.7-1.3 Cr >4.0 mg/dL is automatically assigned a value of 4.0
    
    Bilirubin (mg/dL) Norm = 0.3-1.9
    
    INR Norm 1-2
    
    Sodium (mEq/L) Norm 136-145
    
    FORMULA Per OPTN policy, January 2016 (pages 4–5):
    
    Candidates who are at least 12 years old receive an initial MELD(i) score equal to:
    
    MELD(i) = 0.957 × ln(Cr) + 0.378 × ln(bilirubin) + 1.120 × ln(INR) + 0.643
    
    Then, round to the tenth decimal place and multiply by 10. Maximum MELD = 40.
    
    If MELD(i) > 11, perform additional MELD calculation as follows:
    
    MELD = MELD(i) + 1.32 × (137 – Na) – [ 0.033 × MELD(i) × (137 – Na) ]
    
    Additional rules:
    
    All values in US units (Cr and bilirubin in mg/dL, Na in mEq/L, and INR unitless).
    
    If bilirubin, Cr, or INR is <1.0, use 1.0.
    
    If any of the following is true, use Cr 4.0:
    
          Cr >4.0.
    
          ≥2 dialysis treatments within the prior 7 days.
    
          24 hours of continuous veno-venous hemodialysis (CVVHD) within the prior 7 days.
    
    If Na <125 mmol/L, use 125. If Na >137 mmol/L, use 137.
    
    """
    
    if(pd.isna([Creatinine, Bilirubin, INR, Na]).any()):
        return np.nan
    
    Creatinine = str(Creatinine).replace('<', '').replace('>', '')
    if float(Creatinine) > 4.0:
        Creatinine = 4.0
    elif float(Creatinine) < 1.0:
        Creatinine = 1.0
    elif renal_comorb == 1:
        Creatinine = 4.0
        
    Bilirubin = str(Bilirubin).replace('<', '').replace('>', '')
    if float(Bilirubin) < 1.0:
        Bilirubin = 1.0
        
    INR = str(INR).replace('<', '').replace('>', '')
    if float(INR) < 1.0:
        INR = 1.0
        
    Na = str(Na).replace('<', '').replace('>', '')
    if float(Na) > 137.0:
        Na = 137.0
    elif float(Na) < 125.0:
        Na = 125.0
    
    MELDi = (0.957 * np.log(float(Creatinine))) + (0.378 * np.log(float(Bilirubin))) + (1.120 * np.log(float(INR))) + 0.643
    MELDi = round(MELDi,1)*10.0
    
    if MELDi > 11:
        MELD = MELDi + (1.32 * (137.00 - float(Na))) -  ((0.033 * MELDi) * (137.00 - float(Na)))
    else:
        MELD = MELDi

    return int(round(MELD, 0))

# II. Load and Preprocess data before calculating MELD

In [2]:
pred_df = pd.read_csv('data/main_prospective_val_df_20220805_20220930_20221010.csv')
pred_df_full = pd.read_pickle('data/pred_df_full_20220805_20220930_20221010.pkl')
pred_df_full.iloc[:, 4:]

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (228) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,DepartmentName,LocationAbbreviation,LOS,rel_time,abs_time,AdmissionInstant,AdmissionDateKey,etco2,SBP,DBP,...,SepsisLabel_0.09,SepsisLabel_0.1,SepsisLabel_0.15,SepsisLabel_0.2,SepsisLabel_0.25,SepsisLabel_0.3,SepsisLabel_0.4,SepsisLabel_0.5,Prediction_Time,INR_MELD_Value
0,TH 6 MAIN PERI-OP,Tisch,0.0,0.5,2022-08-03 06:36:00,2022-08-03 06:06:00,20220803,NaN,132.0,76.0,...,0,0,0,0,0,0,0,0,2022-11-21 18:35:13,NaN
1,LB 4900,LB,0.0,0.5,2022-08-24 13:42:00,2022-08-24 13:12:00,20220824,NaN,106.0,85.0,...,0,0,0,0,0,0,0,0,2022-11-21 18:35:13,3.0
2,LB EMERGENCY DEPT,LB,0.0,0.5,2022-08-05 12:37:00,2022-08-05 12:07:00,20220805,NaN,156.0,98.0,...,0,0,0,0,0,0,0,0,2022-11-21 18:35:13,1.3
3,TH 14 EAST,Tisch,0.0,0.5,2022-08-24 17:19:00,2022-08-24 16:49:00,20220824,NaN,113.0,73.0,...,1,1,1,1,1,1,1,0,2022-11-21 18:35:13,1.0
4,LI NW 3 NP 3800,NYUWH,0.0,0.5,2022-09-18 08:16:00,2022-09-18 07:46:00,20220918,NaN,156.0,65.0,...,0,0,0,0,0,0,0,0,2022-11-21 18:35:13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3460290,LB EMERGENCY DEPT,LB,1706.0,1706.5,2022-10-05 21:56:00,2022-07-26 19:26:00,20220726,NaN,115.0,79.0,...,0,0,0,0,0,0,0,0,2022-11-21 18:35:13,NaN
3460291,LB EMERGENCY DEPT,LB,1706.5,1707.0,2022-10-05 22:26:00,2022-07-26 19:26:00,20220726,NaN,115.0,79.0,...,0,0,0,0,0,0,0,0,2022-11-21 18:35:13,NaN
3460292,LB EMERGENCY DEPT,LB,1707.0,1707.5,2022-10-05 22:56:00,2022-07-26 19:26:00,20220726,NaN,115.0,79.0,...,0,0,0,0,0,0,0,0,2022-11-21 18:35:13,NaN
3460293,LB EMERGENCY DEPT,LB,1707.5,1708.0,2022-10-05 23:26:00,2022-07-26 19:26:00,20220726,NaN,115.0,79.0,...,0,0,0,0,0,0,0,0,2022-11-21 18:35:13,NaN


In [4]:
def get_meld_params(pred_df, pred_df_full):
    select_cols = ['ID','abs_time', 'AdmissionInstant', 'rel_time', 'BILIRUBIN TOTAL', 'CREATININE', 'SODIUM', 'INR_MELD_Value']
    pred_df_full = pred_df_full[select_cols]
    
    meld_cols = ['BILIRUBIN TOTAL', 'CREATININE', 'SODIUM', 'INR_MELD_Value']
    for col in meld_cols:
        pred_df_full[col] = pred_df_full[col].replace('CLOTTED', np.nan).replace('NOT CALCULATED', np.nan).replace('').replace('>9.0', '9.0').replace('>9.6', '9.6').replace('<0.7', '0.7')
    pred_df_full['INR_MELD_Value'] = pred_df_full['INR_MELD_Value'].astype(float)
    
    #get the first values as admission indices
    pred_df_full = pred_df_full.sort_values('rel_time', ascending=True).groupby('ID', as_index=False).first(numeric_only=True)
    pred_df_full.rename(columns={'BILIRUBIN TOTAL':'Bilirubin_MELD', 'CREATININE':'Creatinine_MELD',
                            'SODIUM':'Sodium_MELD', 'INR_MELD_Value':'INR_MELD'}, inplace=True)
    pred_df = pred_df.merge(pred_df_full[['ID', 'Bilirubin_MELD', 'Creatinine_MELD', 'Sodium_MELD', 'INR_MELD']], on=['ID'], how='left')
    return pred_df

pred_df = get_meld_params(pred_df, pred_df_full)
pred_df.iloc[:,4:]

<ipython-input-4-285976fada72>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df_full[col] = pred_df_full[col].replace('CLOTTED', np.nan).replace('NOT CALCULATED', np.nan).replace('').replace('>9.0', '9.0').replace('>9.6', '9.6').replace('<0.7', '0.7')
<ipython-input-4-285976fada72>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df_full['INR_MELD_Value'] = pred_df_full['INR_MELD_Value'].astype(float)


,DepartmentName,LocationAbbreviation,LOS,rel_time,abs_time,AdmissionInstant,AdmissionDateKey,etco2,SBP,DBP,...,Immunocompromised_Comorbidity,AKI_Comorbidity,apache_ii_score,apache_ii_Positive,MELD_Score,MELD_Score_Positive,Bilirubin_MELD,Creatinine_MELD,Sodium_MELD,INR_MELD
0,LI NW 1 MB 1400,NYUWH,0.0,0.5,2022-07-24 21:13:00,2022-07-24 20:43:00,20220724,NaN,82.5,50.0,...,0.0,0.0,NaN,0,NaN,0,0.9,1.71,136.0,NaN
1,LI NW 3 HP 3500 MICU,NYUWH,0.0,0.5,2022-07-24 10:07:00,2022-07-24 09:37:00,20220724,NaN,77.0,48.0,...,1.0,1.0,NaN,0,22.0,0,0.5,6.84,134.0,0.95
2,TH KP 13,Tisch,0.0,0.5,2022-08-24 15:06:00,2022-08-24 14:36:00,20220824,33.5,NaN,NaN,...,0.0,0.0,NaN,0,26.0,1,1.2,2.53,134.0,2.00
3,TH EMERGENCY DEPT,Tisch,0.0,0.5,2022-08-24 17:42:00,2022-08-24 17:12:00,20220824,NaN,136.5,70.5,...,0.0,0.0,NaN,0,11.0,0,0.6,0.82,135.0,1.50
4,TH CH EMERG DEPT,Tisch,0.0,0.5,2022-08-25 03:06:00,2022-08-25 02:36:00,20220825,NaN,103.0,86.0,...,0.0,1.0,NaN,0,7.0,0,0.4,0.83,140.0,1.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13811,LI NW 6 HP 6500,NYUWH,0.0,15.5,2022-09-09 07:26:00,2022-09-08 15:56:00,20220908,NaN,90.0,61.0,...,0.0,1.0,NaN,0,NaN,0,NaN,NaN,NaN,NaN
13812,LI NW 5 HP 5500,NYUWH,0.0,16.5,2022-08-23 08:18:00,2022-08-22 15:48:00,20220822,NaN,NaN,NaN,...,0.0,1.0,NaN,0,NaN,0,NaN,NaN,NaN,NaN
13813,LB 4500,LB,0.0,18.5,2022-08-24 09:13:00,2022-08-23 14:43:00,20220823,NaN,90.0,74.0,...,0.0,1.0,NaN,0,NaN,0,NaN,NaN,NaN,NaN
13814,LB 5400,LB,0.0,23.0,2022-09-04 18:09:00,2022-09-03 19:09:00,20220903,NaN,119.0,89.0,...,0.0,1.0,NaN,0,NaN,0,NaN,1.01,155.0,NaN


# III. Calculate MELD Score

In [5]:
pred_df['MELD_Score'] = pred_df.apply(lambda x: calcMELD(x['Creatinine_MELD'], x['Renal_Comorbidity'], x['Bilirubin_MELD'], x['INR_MELD'], x['Sodium_MELD']), axis=1)
pred_df['MELD_Score_Positive'] = pred_df['MELD_Score'].apply(lambda x: 1 if x >= 25 else 0)
pred_df.iloc[:,4:]

,DepartmentName,LocationAbbreviation,LOS,rel_time,abs_time,AdmissionInstant,AdmissionDateKey,etco2,SBP,DBP,...,Immunocompromised_Comorbidity,AKI_Comorbidity,apache_ii_score,apache_ii_Positive,MELD_Score,MELD_Score_Positive,Bilirubin_MELD,Creatinine_MELD,Sodium_MELD,INR_MELD
0,LI NW 1 MB 1400,NYUWH,0.0,0.5,2022-07-24 21:13:00,2022-07-24 20:43:00,20220724,NaN,82.5,50.0,...,0.0,0.0,NaN,0,NaN,0,0.9,1.71,136.0,NaN
1,LI NW 3 HP 3500 MICU,NYUWH,0.0,0.5,2022-07-24 10:07:00,2022-07-24 09:37:00,20220724,NaN,77.0,48.0,...,1.0,1.0,NaN,0,22.0,0,0.5,6.84,134.0,0.95
2,TH KP 13,Tisch,0.0,0.5,2022-08-24 15:06:00,2022-08-24 14:36:00,20220824,33.5,NaN,NaN,...,0.0,0.0,NaN,0,26.0,1,1.2,2.53,134.0,2.00
3,TH EMERGENCY DEPT,Tisch,0.0,0.5,2022-08-24 17:42:00,2022-08-24 17:12:00,20220824,NaN,136.5,70.5,...,0.0,0.0,NaN,0,11.0,0,0.6,0.82,135.0,1.50
4,TH CH EMERG DEPT,Tisch,0.0,0.5,2022-08-25 03:06:00,2022-08-25 02:36:00,20220825,NaN,103.0,86.0,...,0.0,1.0,NaN,0,7.0,0,0.4,0.83,140.0,1.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13811,LI NW 6 HP 6500,NYUWH,0.0,15.5,2022-09-09 07:26:00,2022-09-08 15:56:00,20220908,NaN,90.0,61.0,...,0.0,1.0,NaN,0,NaN,0,NaN,NaN,NaN,NaN
13812,LI NW 5 HP 5500,NYUWH,0.0,16.5,2022-08-23 08:18:00,2022-08-22 15:48:00,20220822,NaN,NaN,NaN,...,0.0,1.0,NaN,0,NaN,0,NaN,NaN,NaN,NaN
13813,LB 4500,LB,0.0,18.5,2022-08-24 09:13:00,2022-08-23 14:43:00,20220823,NaN,90.0,74.0,...,0.0,1.0,NaN,0,NaN,0,NaN,NaN,NaN,NaN
13814,LB 5400,LB,0.0,23.0,2022-09-04 18:09:00,2022-09-03 19:09:00,20220903,NaN,119.0,89.0,...,0.0,1.0,NaN,0,NaN,0,NaN,1.01,155.0,NaN
